In [1]:
import os
import pathlib as Path
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random as rdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import datetime
from tensorflow.keras.applications import VGG19

In [2]:
def read_img(files):
    images = []
    labels = []
    for i in files:
        image = cv2.imread(i)
        image = cv2.resize(image,(150,150))
        images.append(image)
        labels.append(os.path.split(os.path.split(i)[0])[1])
    images = np.array(images)
    return images,labels

In [3]:
c = 0
for i in os.listdir('Mainpath3/Train'):
    for j in os.listdir(os.path.join('Mainpath3/Train',i)):
        c+=1
print('Total no. of image files in Train1: ',c)

Total no. of image files in Train1:  6611


In [4]:
files = tf.io.gfile.glob('Mainpath3/Train/*/*.jpg')

In [5]:
rdm.shuffle(files)

In [6]:
train_data,train_labels = read_img(files)

In [7]:
val_files = tf.io.gfile.glob('Mainpath3/new_val/*/*.jpg')

In [8]:
rdm.shuffle(val_files)

In [9]:
val_data,val_labels = read_img(val_files)

In [10]:
test_files = tf.io.gfile.glob('Mainpath3/Test/*/*.jpg')

In [11]:
test_data,test_labels = read_img(test_files)

In [12]:
categories = {'Normal':0, 'Diabetes':1, 'Glaucoma':2, 'Cataract':3, 'Other':4,'Pathologocal_Myopia':5,'Macular_Deneration':6,'Hypertension':7}

In [13]:
train_mapped_lables = list(map(lambda x: categories.get(x, x), train_labels))

In [14]:
val_mapped_lables = list(map(lambda x: categories.get(x, x), val_labels))
test_mapped_lables = list(map(lambda x: categories.get(x, x), test_labels))

In [15]:
train_arr = np.array(train_mapped_lables)
val_arr = np.array(val_mapped_lables)
test_arr = np.array(test_mapped_lables)

# VGG model


In [16]:
def VGG_model():
    IMAGE_SIZE = (150,150)
    with tf.device('/gpu:0'):
        pretrained_model = tf.keras.applications.VGG19(
            weights='imagenet',
            include_top=False ,
            input_shape=[*IMAGE_SIZE, 3]
        )
        pretrained_model.trainable = False

        
        
        input_layer = tf.keras.layers.Input(shape=(*IMAGE_SIZE, 3), name='input') 
        
        x = pretrained_model(input_layer)

        x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pooling2d')(x)
        x = tf.keras.layers.BatchNormalization()(x)       
        x = tf.keras.layers.Dense(256,activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(128, activation='relu')(x)
        
        output = tf.keras.layers.Dense(units=5, 
                              activation='softmax', 
                              name='output')(x)


        model = Model (input_layer, output)    
        model.compile(optimizer='adam', 
                      loss='sparse_categorical_crossentropy', 
                      metrics=['accuracy'])

        return model

model = VGG_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 150, 150, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 batch_normalization_1 (Bat  (None, 256)               1024  

In [ ]:
run_time = datetime.datetime.now()
lrr = ReduceLROnPlateau(monitor='val_accuracy', patience = 1, verbose=1, factor=0.3, min_lr=0.000001)
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
)

trained = model.fit(
                    train_data,train_arr,
                    validation_data = (val_data,val_arr),
                    batch_size = 32, 
                    epochs=50,
                    callbacks=[lrr, es]
    )

requared_time = datetime.datetime.now() - run_time
print(f'\nRequired time:  {str(requared_time)}\n')

In [ ]:
loss, accuracy = model.evaluate(test_data,test_arr)

In [ ]:
print('loss: ',loss,'  accuracy: ', accuracy)